<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#todos" data-toc-modified-id="todos-1">todos</a></span></li><li><span><a href="#1.-setup" data-toc-modified-id="1.-setup-2">1. setup</a></span></li><li><span><a href="#2.-get-data" data-toc-modified-id="2.-get-data-3">2. get data</a></span><ul class="toc-item"><li><span><a href="#2.A.-create-feature-matrix" data-toc-modified-id="2.A.-create-feature-matrix-3.1">2.A. create feature matrix</a></span></li><li><span><a href="#2.B.-create-label-vector" data-toc-modified-id="2.B.-create-label-vector-3.2">2.B. create label vector</a></span></li><li><span><a href="#2.C.-create-report" data-toc-modified-id="2.C.-create-report-3.3">2.C. create report</a></span></li><li><span><a href="#2.D.-split-data-&amp;-shuffle" data-toc-modified-id="2.D.-split-data-&amp;-shuffle-3.4">2.D. split data &amp; shuffle</a></span></li></ul></li><li><span><a href="#2.-pipeline" data-toc-modified-id="2.-pipeline-4">2. pipeline</a></span><ul class="toc-item"><li><span><a href="#2.A.-metrics-wrap-up" data-toc-modified-id="2.A.-metrics-wrap-up-4.1">2.A. metrics wrap up</a></span><ul class="toc-item"><li><span><a href="#2.A.1.-confusion-matrix-metrics" data-toc-modified-id="2.A.1.-confusion-matrix-metrics-4.1.1">2.A.1. confusion matrix metrics</a></span><ul class="toc-item"><li><span><a href="#true-negative-(TN)" data-toc-modified-id="true-negative-(TN)-4.1.1.1">true negative (TN)</a></span></li><li><span><a href="#false-negative-(FN)" data-toc-modified-id="false-negative-(FN)-4.1.1.2">false negative (FN)</a></span></li><li><span><a href="#true-positive-(TP)" data-toc-modified-id="true-positive-(TP)-4.1.1.3">true positive (TP)</a></span></li><li><span><a href="#false-positive-(FP)" data-toc-modified-id="false-positive-(FP)-4.1.1.4">false positive (FP)</a></span></li></ul></li><li><span><a href="#2.A.2.-metrics" data-toc-modified-id="2.A.2.-metrics-4.1.2">2.A.2. metrics</a></span><ul class="toc-item"><li><span><a href="#precision" data-toc-modified-id="precision-4.1.2.1">precision</a></span></li><li><span><a href="#recall-or-True-Positive-Rate-(TPR)" data-toc-modified-id="recall-or-True-Positive-Rate-(TPR)-4.1.2.2">recall or True Positive Rate (TPR)</a></span></li><li><span><a href="#False-Positive-Rate-(FPR)" data-toc-modified-id="False-Positive-Rate-(FPR)-4.1.2.3">False Positive Rate (FPR)</a></span></li><li><span><a href="#Receiver-Operating-Characteristic-curve-(ROC)" data-toc-modified-id="Receiver-Operating-Characteristic-curve-(ROC)-4.1.2.4">Receiver Operating Characteristic curve (ROC)</a></span></li><li><span><a href="#in-our-case" data-toc-modified-id="in-our-case-4.1.2.5">in our case</a></span></li></ul></li></ul></li><li><span><a href="#2.B.-pipeline-creation" data-toc-modified-id="2.B.-pipeline-creation-4.2">2.B. pipeline creation</a></span></li></ul></li><li><span><a href="#4.-optimization" data-toc-modified-id="4.-optimization-5">4. optimization</a></span><ul class="toc-item"><li><span><a href="#4.A.-gradient-boosting" data-toc-modified-id="4.A.-gradient-boosting-5.1">4.A. gradient boosting</a></span></li><li><span><a href="#4.B.-XGBoost" data-toc-modified-id="4.B.-XGBoost-5.2">4.B. XGBoost</a></span></li></ul></li><li><span><a href="#5.-Evaluation" data-toc-modified-id="5.-Evaluation-6">5. Evaluation</a></span><ul class="toc-item"><li><span><a href="#5.A.-learning-curves" data-toc-modified-id="5.A.-learning-curves-6.1">5.A. learning curves</a></span></li><li><span><a href="#5.B-confusion-matrix" data-toc-modified-id="5.B-confusion-matrix-6.2">5.B confusion matrix</a></span></li><li><span><a href="#5.C-precision-recall-vs-threshold" data-toc-modified-id="5.C-precision-recall-vs-threshold-6.3">5.C precision-recall vs threshold</a></span></li><li><span><a href="#5.D.-ROC-curves" data-toc-modified-id="5.D.-ROC-curves-6.4">5.D. ROC curves</a></span></li></ul></li><li><span><a href="#6.-Summary" data-toc-modified-id="6.-Summary-7">6. Summary</a></span></li></ul></div>

# Project Fatigue
- GitHub [link](https://github.com/romainmartinez/fatigue)

## todos
- keras/tensorflow: [link](https://medium.com/@williamkoehrsen/deep-neural-network-classifier-32c12ff46b6c)
- model optimization
- viz xgboost
- add sex in pipeline
- report html
- feature importance
- learning curves

## 1. setup

In [1]:
# Common imports
import scipy.io as sio
import numpy as np
import warnings
import os

# Figures
OFFLINE = True
if OFFLINE:
    import plotly.offline as py
    py.init_notebook_mode(connected=True)
else:
    import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.rcParams['axes.labelsize'] = 14
# plt.rcParams['xtick.labelsize'] = 12
# plt.rcParams['ytick.labelsize'] = 12
# sns.set_context("notebook", font_scale=1.1)
# sns.set_style("ticks")
# plt.rcParams['pdf.fonttype'] = 42
# plt.rcParams['ps.fonttype'] = 42

# to make this notebook's output stable across runs
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 2. get data

In [2]:
DATA_PATH = os.path.join('.', 'data/')
DATA_FILE = 'DatabaseRPT.mat'
mat = sio.loadmat(os.path.join(DATA_PATH, DATA_FILE))['DataBaseRPT'][0, 0]

- label
    - `Y` (1, 162): 1 = prefatigue, 2 = fatigue

- used features
    - `AllX` (24, 162): 24 (6 DoF x 4 variables) x 162 (81 participants x 2 time points).
    - `CAssignAll` (1, 24): AllX column assignment

    - `Sex` (1, 162)
    - `Endurance` (1, 162)

- not used
    - `BestX` (7, 162): 7 (variables with SRM>0.8) x 162 (81 participants x 2 time points) matrix. Contains data only for the most responsive variables (SRM>0.8).
    - `CAssignBest` (1, 7): BestX column assignment.
    - `Age` (1, 162): too much NaN.
    - `Height` (1, 162): too much NaN.
    - `Weight` (1, 162) Too much NaN.
    - `SubjectKey` (1, 162): useless.
    - `SID` (1, 162): useless.

### 2.A. create feature matrix

In [3]:
col_names = [i[0] for i in mat['CAssignAll'].flatten()]
# find variable with SRM > .8
srm_names = np.array([i[0] for i in mat['CAssignBest'].flatten()])
srm_idx = np.in1d(col_names, srm_names)

# add `AllX`
X = mat['AllX'].T

# add `Sex`
X = np.c_[X, mat['Sex'].T]
col_names.append('Sex')

col_names = np.array(col_names)

### 2.B. create label vector

In [4]:
from sklearn.preprocessing import LabelBinarizer
y = mat['Y'].T
y = LabelBinarizer().fit_transform(y)

### 2.C. create report

In [5]:
import os.path
import pandas as pd

# create dataframe
df = pd.DataFrame(X, columns=col_names)
df['fatigue (label)'] = y

# generate pandas report
REPORT_FILENAME = './pandas_report.html'
if not os.path.isfile(REPORT_FILENAME):
    import pandas_profiling
    report = pandas_profiling.ProfileReport(df)
    report.to_file('./pandas_report.html')

### 2.D. split data & shuffle

In [6]:
# split data & shuffle
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED)

In [7]:
df.head()

,AveElbFlexMean,ROMElbFlexMean,AveElbFlexSD,ROMElbFlexSD,AveShPlaneMean,ROMShPlaneMean,AveShPlaneSD,ROMShPlaneSD,AveShEleMean,ROMShEleMean,...,AveTrYMean,ROMTrYMean,AveTrYSD,ROMTrYSD,AveTrZMean,ROMTrZMean,AveTrZSD,ROMTrZSD,Sex,fatigue (label)
0,105.983144,63.339618,2.219401,1.794134,23.518597,13.624799,1.338393,2.357813,80.274017,2.942674,...,16.382423,13.629558,1.112671,1.977798,-0.550047,2.671822,0.228494,0.711787,2.0,0
1,106.166216,61.875226,2.239310,2.548458,22.851141,11.726534,1.182490,1.401127,77.897777,4.062635,...,16.041630,15.078508,0.910697,1.003788,-0.208865,1.242553,0.541000,0.496108,2.0,1
2,100.914284,60.930029,1.482458,1.167593,27.289316,12.291697,1.223189,2.452705,83.932823,2.886935,...,15.039772,14.311074,0.979720,2.070705,0.096174,0.934767,0.488540,0.317386,2.0,0
3,105.066549,59.524098,2.364182,1.849991,23.083816,7.519415,1.510000,3.503819,78.701025,2.997990,...,16.070120,16.178023,1.418294,3.521951,-0.544067,1.885480,0.610577,0.851287,2.0,1
4,93.259333,81.531138,2.610755,3.632922,42.387152,27.789377,1.605530,2.027628,71.280986,8.000008,...,8.908385,8.815273,0.709582,1.250228,-0.123442,1.065910,0.415764,0.404298,2.0,0


## 2. pipeline

### 2.A. metrics wrap up

#### 2.A.1. confusion matrix metrics

| Predicted | Real: 1 | Real: 0 |
|-----------|---------|---------|
|    1      |  TP     |    FP   |
|    0      |  FN     |    TN   |

##### true negative (TN)
We *correctly* predict that the class is *negative*.

##### false negative (FN)
We *incorrectly* predict that the class is *negative*.

##### true positive (TP)
We *correctly* predict that the class is *positive*.

##### false positive (FP)
We *incorrectly* predict that the class is *positive*.

#### 2.A.2. metrics

##### precision
$$\frac{TP}{\text{n}^o \text{predicted positives}} = \frac{TP}{TP + FP}$$

- high precision = less _false positive_
- _example_: getting $0.8$ precision means that when it claims to detect a label, it is right $80$% of the time

##### recall or True Positive Rate (TPR)
$$\frac{TP}{\text{n}^o \text{actual positives}} = \frac{TP}{TP + FN}$$

- high recall = less _false negative_
- _example_: getting $0.8$ recall means that it detect $80$% of the label

##### False Positive Rate (FPR)
$$\frac{FP}{\text{n}^o \text{actual negatives}} = \frac{FP}{FP + TN}$$

- high FPR = more _negative data points missclassified_
- proportion of negative data points that are mistakenly considered as positive, with respect to all negative data points

##### Receiver Operating Characteristic curve (ROC)
Compute _TPR_ and _FPR_ for many different threshold and plot FPR vs. TPR

##### in our case
We want to optimize our algorithm for the area Under the ROC curve ([wiki](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve), [stackexchange](https://stats.stackexchange.com/a/132832)).

### 2.B. pipeline creation

In [8]:
def get_categorical_cols(X):
    return X[:, col_names == 'Sex']


def get_numerical_cols(X):
    return X[:, col_names != 'Sex']


def get_high_srm_cols(X, srm_idx=srm_idx):
    return X[:, srm_idx]

In [9]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

# categorical
from sklearn.preprocessing import OneHotEncoder

# numerical
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures

pipeline_categorical = Pipeline([
    ('selector', FunctionTransformer(get_categorical_cols, validate=False)),
    ('encoder', OneHotEncoder(sparse=False))
])

pipeline_numerical = Pipeline([
    ('selector', FunctionTransformer(get_numerical_cols, validate=False)),
    ('selecthighsrm', FunctionTransformer(get_high_srm_cols, validate=False)),
    ('scale', StandardScaler()),
    ('polyfeatures', PolynomialFeatures(degree=2))
])

pipeline_preprocessing = FeatureUnion([
    ('categorical', pipeline_categorical),
    ('numerical', pipeline_numerical)
])

In [10]:
from sklearn.model_selection import GridSearchCV
import pprint


class GridSearchWrapper:
    def __init__(self, base_pipeline, model, param_grid, scoring):
        self.base_pipeline = base_pipeline
        self.model = model
        self.param_grid = param_grid
        self.scoring = scoring
    
    def fit(self, X, y):
        pipeline = Pipeline([
            ('preprocessing', self.base_pipeline),
            ('pca', PCA(n_components=0.99, svd_solver='full')),
            ('classifier', self.model)
        ])
        
        self.grid_search = GridSearchCV(pipeline,
                                        self.param_grid,
                                        scoring=self.scoring,
                                        cv=5,
                                        n_jobs=-1,
                                        verbose=1)
        self.grid_search.fit(X, y)
    
    def print_scores(self):
        print(f'best score ({self.scoring}): {self.grid_search.best_score_:.3f}')
        print('\nbest params:')
        pprint.pprint(self.grid_search.best_params_)

In [11]:
param_grid = {
    'preprocessing__numerical__selecthighsrm': [None, FunctionTransformer(get_high_srm_cols, validate=False)],
    'preprocessing__numerical__polyfeatures': [None, PolynomialFeatures(degree=1), PolynomialFeatures(degree=2),
                                               PolynomialFeatures(degree=3)],
    'preprocessing__numerical__scale': [None, StandardScaler()],
    'pca': [None, PCA(n_components=0.99, svd_solver='full')]
}

scoring = 'roc_auc'
model = {}

## 4. optimization

### 4.A. gradient boosting

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

model_param = {
    'learning_rate': 0.001,
    'max_depth': 1,
    'max_features': 0.75,
    'min_samples_leaf': 7,
    'min_samples_split': 16,
    'n_estimators': 100,
    'subsample': 0.3,
    'random_state': RANDOM_SEED
}

wrapper = GridSearchWrapper(
    base_pipeline=pipeline_preprocessing,
    model=GradientBoostingClassifier(**model_param),
    param_grid=param_grid,
    scoring=scoring)

wrapper.fit(X_train, y_train.ravel())

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:    5.7s finished


In [13]:
wrapper.print_scores()
model['gradboost'] = wrapper.grid_search.best_estimator_

best score (roc_auc): 0.912

best params:
{'pca': PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=None,
  svd_solver='full', tol=0.0, whiten=False),
 'preprocessing__numerical__polyfeatures': None,
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': FunctionTransformer(accept_sparse=False,
          func=<function get_high_srm_cols at 0x7fcf9de1abf8>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)}


### 4.B. XGBoost

https://jessesw.com/XG-Boost/

In [14]:
from xgboost import XGBClassifier

model_param = {
    'learning_rate': 0.001,
    'max_depth': 1,
    'max_features': 0.75,
    'min_samples_leaf': 7,
    'min_samples_split': 16,
    'n_estimators': 100,
    'subsample': 0.3,
    'random_state': RANDOM_SEED
}

warnings.simplefilter('ignore')
wrapper = GridSearchWrapper(
    base_pipeline=pipeline_preprocessing,
    model=XGBClassifier(**model_param),
    param_grid=param_grid,
    scoring=scoring)

wrapper.fit(X_train, y_train.ravel())
warnings.simplefilter('default')

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:    5.0s finished


In [15]:
wrapper.print_scores()
model['xgboost'] = wrapper.grid_search.best_estimator_

best score (roc_auc): 0.902

best params:
{'pca': PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=None,
  svd_solver='full', tol=0.0, whiten=False),
 'preprocessing__numerical__polyfeatures': None,
 'preprocessing__numerical__scale': None,
 'preprocessing__numerical__selecthighsrm': FunctionTransformer(accept_sparse=False,
          func=<function get_high_srm_cols at 0x7fcf9de1abf8>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)}


## 5. Evaluation

In [16]:
from sklearn.metrics import precision_recall_curve, roc_curve, classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix

class Evaluate:
    def __init__(self, model):
        self.model = model
        
    def classification_report(self):
        y_scores = self.grid_search.predict(X)
        report = classification_report(y, y_scores)
        print(report)
        
        self.confusion_matrix(X, y)
        self.precision_recall_vs_threshold(X, y)
        self.roc_curve(X, y)
        
    def confusion_matrix(self, X, y, percentage=True):   
        y_pred = self.grid_search.best_estimator_.predict(X)
        cm = confusion_matrix(y, y_pred)
        if percentage:
            cm = cm / cm.sum(axis=1)[:, np.newaxis] * 100
            
        plt.figure(figsize=(8, 4))
        sns.heatmap(cm, annot=True, cmap='RdYlGn')
        plt.title('Confusion matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
    
    def precision_recall_vs_threshold(self, X, y):
        y_scores = self.grid_search.predict_proba(X)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y, y_scores)
        plt.figure(figsize=(8, 8))
        plt.title("Precision and Recall Scores as a function of the decision threshold")
        plt.plot(thresholds, precision[:-1], "b--", label="Precision")
        plt.plot(thresholds, recall[:-1], "g-", label="Recall")
        plt.ylabel("Score")
        plt.xlabel("Decision Threshold")
        plt.legend(loc='best')
        
    def roc_curve(self, X, y):
        y_scores = self.grid_search.predict_proba(X)[:, 1]
        fpr, tpr, auc_thresholds = roc_curve(y, y_scores)
        roc_auc = roc_auc_score(y, y_scores)
        plt.figure(figsize=(8,8))
        plt.title(f'ROC Curve (roc_auc = {roc_auc:.3f})')
        plt.plot(fpr, tpr, linewidth=2)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.axis([-0.005, 1, 0, 1.005])
        plt.xticks(np.arange(0,1, 0.05), rotation=90)
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate (Recall)")
        
    def decision_threshold(self, X, y, t=0.5):
        y_scores = self.grid_search.predict_proba(X)[:, 1]
        y_scores_adj = [1 if y >= t else 0 for y in y_scores]

        cm = confusion_matrix(y, y_scores_adj)
        sns.heatmap(cm, annot=True)
        plt.show()

        precision, recall, thresholds = precision_recall_curve(y, y_scores)
        plt.figure(figsize=(8,8))
        plt.title("Precision and Recall curve (^=current threshold)")
        plt.step(recall, precision, color='b', alpha=0.2,
                 where='post')
        plt.fill_between(recall, precision, step='post', alpha=0.2,
                         color='b')
        plt.ylim([0.5, 1.01]);
        plt.xlim([0.5, 1.01]);
        plt.xlabel('Recall');
        plt.ylabel('Precision');

        # plot the current threshold on the line
        close_default_clf = np.argmin(np.abs(thresholds - t))
        plt.plot(recall[close_default_clf], precision[close_default_clf], '^', c='k',
                markersize=15)    

### 5.A. learning curves

In [17]:
y_pred = model['xgboost'].predict(X_test)

/home/romain/.local/share/virtualenvs/fatigue-WgHAVgza/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



In [141]:
from sklearn.model_selection import learning_curve


def plot_learning_curve(
        estimator,
        X,
        y,
        scoring='roc_auc',
        cv=None,
        n_jobs=1,
        train_sizes=np.linspace(.1, 1.0, 5),
):
    warnings.simplefilter('ignore')  # ignore scikit-learn warnings
    train_sizes, train_scores, test_scores = learning_curve(
        estimator,
        X,
        y,
        scoring=scoring,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes)
    warnings.simplefilter('default')

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)

    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    trace = []

    # training mean
    trace.append(go.Scatter(
        x=train_sizes,
        y=train_scores_mean,
        marker=dict(color='red'),
        name='Training score'))

    # training std
    trace.append(go.Scatter(
        x=train_sizes,
        y=train_scores_mean + train_scores_std,
        mode='lines',
        line=dict(color='red', width=1),
        showlegend=False))

    trace.append(go.Scatter(
        x=train_sizes,
        y=train_scores_mean - train_scores_std,
        mode='lines',
        line=dict(color='red', width=1),
        fill='tonexty',
        showlegend=False))

    # test mean
    trace.append(go.Scatter(
        x=train_sizes,
        y=test_scores_mean,
        marker=dict(color='green'),
        name='Cross-validation score'))

    # test std
    trace.append(go.Scatter(
        x=train_sizes,
        y=test_scores_mean + test_scores_std,
        mode='lines',
        line=dict(color='green', width=1),
        showlegend=False))

    trace.append(go.Scatter(
        x=train_sizes,
        y=test_scores_mean - test_scores_std,
        mode='lines',
        line=dict(color='green', width=1),
        fill='tonexty',
        showlegend=False))

    data = [itrace for itrace in trace]
    layout = go.Layout(
        title='Learning curves',
        xaxis=dict(title='Training examples'),
        yaxis=dict(title='Score (ROC AUC)'),
        hovermode='closest')
    fig = dict(data=data, layout=layout)
    return fig

In [142]:
l = plot_learning_curve(
    estimator=model['xgboost'],
    X=X,
    y=y.ravel(),
    scoring=scoring,
    cv=5,
    train_sizes=np.linspace(.1, 1.0, 10))

py.iplot(l)

### 5.B confusion matrix

In [41]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[12,  5],
       [ 5, 11]])

In [123]:
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(y_test, y_pred, in_percentage=True):
    cm = confusion_matrix(y_test, y_pred)
    if in_percentage:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    colorscale = [[0, '#2d3436'], [1, '#dfe6e9']]

    fig = ff.create_annotated_heatmap(
        z=cm.astype(int),
        x=['True', 'False'],
        y=['True', 'False'],
        showscale=True,
        zmin=0,
        zmax=100,
        colorscale=colorscale,
        colorbar=dict(title='Percentage', titleside='right'))

    # we must alter fig directly instead of creating a layout dict
    fig['layout']['title'] = 'Confusion matrix'
    fig['layout']['xaxis']['title'] = 'Predicted values'
    fig['layout']['xaxis']['side'] = 'bottom'
    fig['layout']['yaxis']['title'] = 'True values'
    fig['layout']['yaxis']['autorange'] = 'reversed'
    fig['layout']['hovermode'] = 'closest'
    return fig

In [124]:
m = plot_confusion_matrix(y_test=y_test, y_pred=y_pred)

py.iplot(m)

### 5.C precision-recall vs threshold

### 5.D. ROC curves

## 6. Summary